In [69]:
import sqlite3
import pandas as pd

# Create your connection.
from twitter.nlp_util import process_tweet

cnx = sqlite3.connect('db.sqlite3')

df = pd.read_sql_query(
    "SELECT id, conversation_id, created_at, text, author_id,in_reply_to_user_id FROM delab_timeline WHERE lang='en'",
    cnx)

df.head(3)


,id,conversation_id,created_at,text,author_id,in_reply_to_user_id
0,1,1435703745304612870,2021-09-08 20:37:01,RT @OregonOEM: 🚩🌩🔥 Red Flag and #FireWeather w...,14838508,NaN
1,2,1435664738353098756,2021-09-08 18:02:01,It's #NationalPreparednessMonth. Help ensure t...,14838508,NaN
2,3,1435663883595706370,2021-09-08 17:58:37,RT @OregonOEM: Oregon is still recovering from...,14838508,NaN


Now we are importing the libraries.

The first idea was to follow lda https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

but turns out it is to little information

In [70]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

nlp = spacy.load("en_core_web_lg")

Showing some descriptive statistics for the author corpus downloaded from twitter.

In [72]:
df.author_id.nunique() # the number of unique authors in the dataset

69

In [74]:
df_reduced = df[["author_id", "text", "id"]]
#df_reduced = df_reduced.groupby('author_id')
# df_reduced.count()

df_reshaped = df_reduced.pivot(index="id",columns="author_id", values="text")
mask = 400 > df_reshaped.nunique()
mask = mask[mask == True]
df_reshaped.drop(columns=mask.index,inplace=True)
df_reshaped.nunique() # the number of tweets of the authors that have more then 400 tweets

author_id
16558158               478
18616003               946
26998226               469
382814447              446
1005470991668084736    445
1106611172462219265    491
1162371171805011968    447
1239172010363826183    427
1292908140975943681    414
1402252385427222528    414
1403930956428460035    441
dtype: int64

The following takes the pandas dataframe and converts it to a dictionary with the author ids as keys and the twitter
corpora as values.

In [82]:
#df_reshaped.shape
author_corpora = df_reshaped.to_dict(orient="series")
for author_id, tweets in author_corpora.items():
    author_corpora[author_id] = tweets.dropna()

example_corpus = author_corpora[next(iter(author_corpora))]
example_corpus

id
8247                                   @ncreen_same ouch!
8248    That last tweet got some responses from spambo...
8249    'virtually' virtually means: real\n\nSo it sho...
8250                               Sounds great on paper!
8251    Any .com.au registrar recommendations? So far ...
                              ...                        
8742    RT @paydirtapp: Check out our Free Invoice Cre...
8743    @taitems @mmilo yeah nice site, suggestion: ht...
8744    RT @MichaelFHansen: Zendesk eyes Southeast Asi...
8745    While LinkedIn has been changing drastically o...
8746    Another LinkedIn email fail… now they want me ...
Name: 16558158, Length: 478, dtype: object

Now we are ready to analyze the corpora of the authors tweets. We do one example with the first corpus

In [87]:
pre_processed_corpus = []
for tweet in example_corpus:
    pre_processed_corpus.append(process_tweet(tweet)) # here we use tweet_tokenizer

pre_processed_corpus

[['ouch'],
 ['last', 'tweet', 'got', 'respons', 'spambot'],
 ['virtual', 'virtual', 'mean', 'real', 'virtual', 'realiti', 'virtual'],
 ['sound', 'great', 'paper'],
 ['com.au',
  'registrar',
  'recommend',
  'far',
  "i'v",
  'left',
  'planetdomain',
  'crazydomain',
  'scream'],
 ['everybodi', 'make', 'mistak', 'last', 'mistak', 'made', 'wow', 'epicfail'],
 ['complaints@crazydomains.com.au', "i'v", 'told', 'write', '2', 'reason'],
 ['zero', 'annoy', 'zilch', 'nada'],
 ['mentor', 'paid', 'free'],
 ['may', 'mean', 'might', 'stop', 'get', 'monthli', 'invoic'],
 ['happen', 'well', 'sure', 'coincid'],
 ['accident',
  'give',
  'peopl',
  '16',
  'day',
  'extra',
  'free',
  'trial',
  'creat',
  'lot',
  'support',
  'request'],
 ['surpris',
  'ask',
  'rate',
  'support',
  'know',
  "i'm",
  'total',
  'outrag',
  'scale',
  '1',
  '10'],
 ['realli', 'happen', 'someon', 'play', 'ukelel', 'outsid', 'offic'],
 ['quick',
  'poll',
  'headstart',
  'event',
  'made',
  'return',
  'would',

In [90]:
pre_processed_corpus = []
for tweet in example_corpus:
    doc = nlp(tweet)
    sentences = list(doc.sents)
    print(sentences[0].ents)


[]
[]
[]
[]
[]
[]
[]
[zero]
[]
[0, http://t.co/K8cPzW6qPk]
[]
[16 days]
[]
[]
[RT @HeadStartAus]
[@alandownie]
[]
[weeks ago]
[Australians, Australian, Europe]
[]
[]
[@jakeisonline]
[@alandownie @rioter @ncreen_same]
[@alandownie @rioter]
[NoSQL, NoArray]
[a few seconds]
[at least a week]
[One, 24 hours]
[]
[7487944]
[]
[]
[Flash, @alandownie]
[@alandownie @skwashd]
[SQL]
[one]
[]
[]
[@gpdawson @runkeeper, SOS]
[@alandownie @bigyahu, 10%, the last month]
[the half million]
[BugHerd, 500,000]
[today]
[NBN, Australia]
[@omnyapp]
[]
[]
[@garyvee]
[]
[]
[]
[]
[]
[Uncategorized]
[Years]
[Daily, US]
[@alandownie @ncreen_same @therealdevgeeks]
[RT @alandownie]
[first]
[@zujiaustralia, 28th Mar 2013, 12-18 weeks]
[140]
[234272]
[Today]
[@theheraldsun]
[]
[]
[Europe]
[]
[RT]
[]
[@jessemcnelis]
[@jessemcnelis]
[]
[]
[RT @thissink]
[RT @damonagle]
[RT @bugherd]
[Peter Mac, @charityvideoau]
[Dogbolter, @matildabaybeer, #]
[Lager Jack, 3, @untappd]
[last holiday, Heineken, @heineken]
[Anheuser-Busc